In [1]:
# 1.  Napisz program w Pythonie, który pobiera dane json i zwraca plik CSV z danymi.
# 2.  Wgraj wygenerowany plik CSV do Google Sheets.
# 3.  W nowym arkuszu stwórz tabelę przestawną, która przedstawia ile meteorów danej klasy spadło w danym roku.

In [1]:
import urllib.request
import json, csv
import pandas as pd
import os
from googleapiclient.http import MediaFileUpload
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import numpy as np


gauth = GoogleAuth()
gauth.LocalWebserverAuth() 
drive = GoogleDrive(gauth)



Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=505264712938-seano1obgad976pfvg351a5s4oc03647.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [2]:
def json_to_csv(link: str, name:str):
    data = dict()
    with urllib.request.urlopen(link) as url:
        data = json.loads(url.read().decode())
        #print(data)
    with open(f'{name}.json', 'w', ) as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    new_data = list()
    for d in data:
        new_d = dict()
        for key, value in d.items():
            if not isinstance(value, dict):
                new_d[key] = value
            else:
                for k, v in value.items():
                    new_d[key + "_" + k] = v
            new_data.append(new_d)
    df = pd.read_json('data.json')
    df = pd.DataFrame(new_data)
    df.to_csv(rf'{name}.csv', index = None)
    return df

In [3]:
df = json_to_csv("https://data.nasa.gov/resource/gh4g-9sfh.json", 'data')

In [5]:
pivot_table = df.groupby(["year", "recclass"])["id"].count()#.to_csv("pivot_table.csv")

In [19]:
pivot_table

year                     recclass             
0861-01-01T00:00:00.000  L6                       10
0921-01-01T00:00:00.000  Stone-uncl                9
1400-01-01T00:00:00.000  Iron, IID                10
1491-01-01T00:00:00.000  Stone-uncl               10
1492-01-01T00:00:00.000  LL6                      10
                                                  ..
2011-01-01T00:00:00.000  L6                       30
                         Martian (shergottite)    10
2012-01-01T00:00:00.000  C                        12
                         L6                       12
2013-01-01T00:00:00.000  LL5                      10
Name: id, Length: 816, dtype: int64

In [44]:
def csv_to_google_sheets(file_path:str, file_name:str, google_folder:list = None):
    if not os.path.exists(file_path):
        print(f'{file_path} not found.')
        return
    try:
        file = drive.CreateFile({"mimeType":"text/csv", "parents":[{"kind": "drive#fileLink", "id": google_folder}]})
        file.SetContentFile(f'{file_name}.csv')
        file.Upload({'convert': True})
        print(file)
        return file
    except Exception as exp:
        print(exp)
        return    
    

In [56]:
csv_to_google_sheets('data.csv', 'data', '18xiAQC_UQxb0bhi3NNPDevBKlUztV5fl')
csv_to_google_sheets('pivot_table.csv', 'pivot_table', '18xiAQC_UQxb0bhi3NNPDevBKlUztV5fl')

GoogleDriveFile({'mimeType': 'application/vnd.google-apps.spreadsheet', 'parents': [{'kind': 'drive#parentReference', 'id': '18xiAQC_UQxb0bhi3NNPDevBKlUztV5fl', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1bGnsILBZWzWhBGx8PDCbUp05Lm8Cu1nRsZMQJyR47EI/parents/18xiAQC_UQxb0bhi3NNPDevBKlUztV5fl', 'parentLink': 'https://www.googleapis.com/drive/v2/files/18xiAQC_UQxb0bhi3NNPDevBKlUztV5fl', 'isRoot': False}], 'title': 'data', 'kind': 'drive#file', 'id': '1bGnsILBZWzWhBGx8PDCbUp05Lm8Cu1nRsZMQJyR47EI', 'etag': '"MTY1MTQ4MDc1MzMyMw"', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1bGnsILBZWzWhBGx8PDCbUp05Lm8Cu1nRsZMQJyR47EI', 'webContentLink': 'https://drive.google.com/uc?id=1bGnsILBZWzWhBGx8PDCbUp05Lm8Cu1nRsZMQJyR47EI&export=download', 'alternateLink': 'https://docs.google.com/spreadsheets/d/1bGnsILBZWzWhBGx8PDCbUp05Lm8Cu1nRsZMQJyR47EI/edit?usp=drivesdk', 'embedLink': 'https://docs.google.com/spreadsheets/d/1bGnsILBZWzWhBGx8PDCbUp05Lm8Cu1nRsZMQJyR47EI/htmlembed?ouid=10776862

GoogleDriveFile({'mimeType': 'application/vnd.google-apps.spreadsheet', 'parents': [{'kind': 'drive#parentReference', 'id': '18xiAQC_UQxb0bhi3NNPDevBKlUztV5fl', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1lLXxxbjjqWmPGUImkRUh4aU2q4KBUFCP6l_6_V2dzCs/parents/18xiAQC_UQxb0bhi3NNPDevBKlUztV5fl', 'parentLink': 'https://www.googleapis.com/drive/v2/files/18xiAQC_UQxb0bhi3NNPDevBKlUztV5fl', 'isRoot': False}], 'title': 'pivot_table', 'kind': 'drive#file', 'id': '1lLXxxbjjqWmPGUImkRUh4aU2q4KBUFCP6l_6_V2dzCs', 'etag': '"MTY1MTQ4MDc1NzI3Mw"', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1lLXxxbjjqWmPGUImkRUh4aU2q4KBUFCP6l_6_V2dzCs', 'webContentLink': 'https://drive.google.com/uc?id=1lLXxxbjjqWmPGUImkRUh4aU2q4KBUFCP6l_6_V2dzCs&export=download', 'alternateLink': 'https://docs.google.com/spreadsheets/d/1lLXxxbjjqWmPGUImkRUh4aU2q4KBUFCP6l_6_V2dzCs/edit?usp=drivesdk', 'embedLink': 'https://docs.google.com/spreadsheets/d/1lLXxxbjjqWmPGUImkRUh4aU2q4KBUFCP6l_6_V2dzCs/htmlembed?ouid=1

https://drive.google.com/drive/folders/18xiAQC_UQxb0bhi3NNPDevBKlUztV5fl?usp=sharing

Additional for data visualization 

In [21]:
nasa_data = pd.read_csv('data.csv')
print(nasa_data.columns)
nasa_data.head()
np.shape(nasa_data)


Index(['name', 'id', 'nametype', 'recclass', 'mass', 'fall', 'year', 'reclat',
       'reclong', 'geolocation_latitude', 'geolocation_longitude',
       ':@computed_region_cbhk_fwbd', ':@computed_region_nnqa_25f4'],
      dtype='object')


(10202, 13)

In [22]:
print(nasa_data.head())

     name  id nametype recclass  mass  fall                     year  reclat  \
0  Aachen   1    Valid       L5  21.0  Fell  1880-01-01T00:00:00.000  50.775   
1  Aachen   1    Valid       L5  21.0  Fell  1880-01-01T00:00:00.000  50.775   
2  Aachen   1    Valid       L5  21.0  Fell  1880-01-01T00:00:00.000  50.775   
3  Aachen   1    Valid       L5  21.0  Fell  1880-01-01T00:00:00.000  50.775   
4  Aachen   1    Valid       L5  21.0  Fell  1880-01-01T00:00:00.000  50.775   

   reclong  geolocation_latitude  geolocation_longitude  \
0  6.08333                50.775                6.08333   
1  6.08333                50.775                6.08333   
2  6.08333                50.775                6.08333   
3  6.08333                50.775                6.08333   
4  6.08333                50.775                6.08333   

   :@computed_region_cbhk_fwbd  :@computed_region_nnqa_25f4  
0                          NaN                          NaN  
1                          NaN          

In [23]:
# we can drop those two attributtes since they are filled with NaN values   
nasa_data = nasa_data.drop([':@computed_region_cbhk_fwbd', ':@computed_region_nnqa_25f4'], axis =1)

In [24]:
nasa_data.head()

,name,id,nametype,recclass,mass,fall,year,reclat,reclong,geolocation_latitude,geolocation_longitude
0,Aachen,1,Valid,L5,21.0,Fell,1880-01-01T00:00:00.000,50.775,6.08333,50.775,6.08333
1,Aachen,1,Valid,L5,21.0,Fell,1880-01-01T00:00:00.000,50.775,6.08333,50.775,6.08333
2,Aachen,1,Valid,L5,21.0,Fell,1880-01-01T00:00:00.000,50.775,6.08333,50.775,6.08333
3,Aachen,1,Valid,L5,21.0,Fell,1880-01-01T00:00:00.000,50.775,6.08333,50.775,6.08333
4,Aachen,1,Valid,L5,21.0,Fell,1880-01-01T00:00:00.000,50.775,6.08333,50.775,6.08333


In [25]:
#there are not a lot of NaN values so we can delete rows with NaN values
print(nasa_data.isnull().sum())

name                       0
id                         0
nametype                   0
recclass                   0
mass                     254
fall                       0
year                       9
reclat                    84
reclong                   84
geolocation_latitude      84
geolocation_longitude     84
dtype: int64


In [26]:
new_nasa_data = nasa_data.dropna()
new_nasa_data['Lat, Long'] = new_nasa_data.geolocation_latitude.map(str) +', '+  new_nasa_data.geolocation_longitude.map(str)
new_nasa_data.to_csv(r'nasa_data.csv', index = None)

C:\Users\alase\AppData\Local\Temp/ipykernel_1456/4169302706.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_nasa_data['Lat, Long'] = new_nasa_data.geolocation_latitude.map(str) +', '+  new_nasa_data.geolocation_longitude.map(str)


In [27]:
new_nasa_data.head()

,name,id,nametype,recclass,mass,fall,year,reclat,reclong,geolocation_latitude,geolocation_longitude,"Lat, Long"
0,Aachen,1,Valid,L5,21.0,Fell,1880-01-01T00:00:00.000,50.775,6.08333,50.775,6.08333,"50.775, 6.08333"
1,Aachen,1,Valid,L5,21.0,Fell,1880-01-01T00:00:00.000,50.775,6.08333,50.775,6.08333,"50.775, 6.08333"
2,Aachen,1,Valid,L5,21.0,Fell,1880-01-01T00:00:00.000,50.775,6.08333,50.775,6.08333,"50.775, 6.08333"
3,Aachen,1,Valid,L5,21.0,Fell,1880-01-01T00:00:00.000,50.775,6.08333,50.775,6.08333,"50.775, 6.08333"
4,Aachen,1,Valid,L5,21.0,Fell,1880-01-01T00:00:00.000,50.775,6.08333,50.775,6.08333,"50.775, 6.08333"


In [45]:
pivot_table = new_nasa_data.groupby(["recclass", "year"])["id"].count().sort_index(ascending=True).to_csv("pivot_table.csv")

In [46]:
csv_to_google_sheets('nasa_data.csv', 'nasa_data', '18xiAQC_UQxb0bhi3NNPDevBKlUztV5fl')

GoogleDriveFile({'mimeType': 'application/vnd.google-apps.spreadsheet', 'parents': [{'kind': 'drive#parentReference', 'id': '18xiAQC_UQxb0bhi3NNPDevBKlUztV5fl', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1fJY4i84-KD0Tdo9NGBUROkHFxkNZpk3SAN5fyl4jyJM/parents/18xiAQC_UQxb0bhi3NNPDevBKlUztV5fl', 'parentLink': 'https://www.googleapis.com/drive/v2/files/18xiAQC_UQxb0bhi3NNPDevBKlUztV5fl', 'isRoot': False}], 'title': 'nasa_data', 'kind': 'drive#file', 'id': '1fJY4i84-KD0Tdo9NGBUROkHFxkNZpk3SAN5fyl4jyJM', 'etag': '"MTY1MTc2OTA5MDM2OA"', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1fJY4i84-KD0Tdo9NGBUROkHFxkNZpk3SAN5fyl4jyJM', 'webContentLink': 'https://drive.google.com/uc?id=1fJY4i84-KD0Tdo9NGBUROkHFxkNZpk3SAN5fyl4jyJM&export=download', 'alternateLink': 'https://docs.google.com/spreadsheets/d/1fJY4i84-KD0Tdo9NGBUROkHFxkNZpk3SAN5fyl4jyJM/edit?usp=drivesdk', 'embedLink': 'https://docs.google.com/spreadsheets/d/1fJY4i84-KD0Tdo9NGBUROkHFxkNZpk3SAN5fyl4jyJM/htmlembed?ouid=107

GoogleDriveFile({'mimeType': 'application/vnd.google-apps.spreadsheet', 'parents': [{'kind': 'drive#parentReference', 'id': '18xiAQC_UQxb0bhi3NNPDevBKlUztV5fl', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1fJY4i84-KD0Tdo9NGBUROkHFxkNZpk3SAN5fyl4jyJM/parents/18xiAQC_UQxb0bhi3NNPDevBKlUztV5fl', 'parentLink': 'https://www.googleapis.com/drive/v2/files/18xiAQC_UQxb0bhi3NNPDevBKlUztV5fl', 'isRoot': False}], 'title': 'nasa_data', 'kind': 'drive#file', 'id': '1fJY4i84-KD0Tdo9NGBUROkHFxkNZpk3SAN5fyl4jyJM', 'etag': '"MTY1MTc2OTA5MDM2OA"', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1fJY4i84-KD0Tdo9NGBUROkHFxkNZpk3SAN5fyl4jyJM', 'webContentLink': 'https://drive.google.com/uc?id=1fJY4i84-KD0Tdo9NGBUROkHFxkNZpk3SAN5fyl4jyJM&export=download', 'alternateLink': 'https://docs.google.com/spreadsheets/d/1fJY4i84-KD0Tdo9NGBUROkHFxkNZpk3SAN5fyl4jyJM/edit?usp=drivesdk', 'embedLink': 'https://docs.google.com/spreadsheets/d/1fJY4i84-KD0Tdo9NGBUROkHFxkNZpk3SAN5fyl4jyJM/htmlembed?ouid=107

In [47]:
csv_to_google_sheets('pivot_table.csv', 'pivot_table', '18xiAQC_UQxb0bhi3NNPDevBKlUztV5fl')

GoogleDriveFile({'mimeType': 'application/vnd.google-apps.spreadsheet', 'parents': [{'kind': 'drive#parentReference', 'id': '18xiAQC_UQxb0bhi3NNPDevBKlUztV5fl', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1TfFvJ2x5gMSPz9NKPoOFU1z5250XtMbUYkp0-6gnKNA/parents/18xiAQC_UQxb0bhi3NNPDevBKlUztV5fl', 'parentLink': 'https://www.googleapis.com/drive/v2/files/18xiAQC_UQxb0bhi3NNPDevBKlUztV5fl', 'isRoot': False}], 'title': 'pivot_table', 'kind': 'drive#file', 'id': '1TfFvJ2x5gMSPz9NKPoOFU1z5250XtMbUYkp0-6gnKNA', 'etag': '"MTY1MTc2OTEzNzA2NQ"', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1TfFvJ2x5gMSPz9NKPoOFU1z5250XtMbUYkp0-6gnKNA', 'webContentLink': 'https://drive.google.com/uc?id=1TfFvJ2x5gMSPz9NKPoOFU1z5250XtMbUYkp0-6gnKNA&export=download', 'alternateLink': 'https://docs.google.com/spreadsheets/d/1TfFvJ2x5gMSPz9NKPoOFU1z5250XtMbUYkp0-6gnKNA/edit?usp=drivesdk', 'embedLink': 'https://docs.google.com/spreadsheets/d/1TfFvJ2x5gMSPz9NKPoOFU1z5250XtMbUYkp0-6gnKNA/htmlembed?ouid=1

GoogleDriveFile({'mimeType': 'application/vnd.google-apps.spreadsheet', 'parents': [{'kind': 'drive#parentReference', 'id': '18xiAQC_UQxb0bhi3NNPDevBKlUztV5fl', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1TfFvJ2x5gMSPz9NKPoOFU1z5250XtMbUYkp0-6gnKNA/parents/18xiAQC_UQxb0bhi3NNPDevBKlUztV5fl', 'parentLink': 'https://www.googleapis.com/drive/v2/files/18xiAQC_UQxb0bhi3NNPDevBKlUztV5fl', 'isRoot': False}], 'title': 'pivot_table', 'kind': 'drive#file', 'id': '1TfFvJ2x5gMSPz9NKPoOFU1z5250XtMbUYkp0-6gnKNA', 'etag': '"MTY1MTc2OTEzNzA2NQ"', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1TfFvJ2x5gMSPz9NKPoOFU1z5250XtMbUYkp0-6gnKNA', 'webContentLink': 'https://drive.google.com/uc?id=1TfFvJ2x5gMSPz9NKPoOFU1z5250XtMbUYkp0-6gnKNA&export=download', 'alternateLink': 'https://docs.google.com/spreadsheets/d/1TfFvJ2x5gMSPz9NKPoOFU1z5250XtMbUYkp0-6gnKNA/edit?usp=drivesdk', 'embedLink': 'https://docs.google.com/spreadsheets/d/1TfFvJ2x5gMSPz9NKPoOFU1z5250XtMbUYkp0-6gnKNA/htmlembed?ouid=1